In [ ]:
import os
from io import BytesIO
import pickle
import torch
import tensorflow.compat.v1 as tf

# tf.enable_eager_execution()

# CONFIG
data_path = os.getcwd() + "/data/training/"


In [ ]:
def load_blob(blob_idx):
    with open(f'{data_path}FRONT/blob_0_frame_0.pickle', 'rb') as f:
        # Load the 'data' dictionary using the highest protocol available.
        return pickle.load(f)

In [ ]:
%%timeit
# import time
# start = time.time()
blob = load_blob(1)
# end = time.time()
# print(end-start)

In [ ]:
blob.keys()

In [ ]:
import numpy as np

# torch.tensor(np.array(bytearray(blob[0]['SIDE_RIGHT']['image']))).shape
x = tf.image.decode_jpeg(blob[0]['SIDE_RIGHT']['image'])


In [ ]:
torch.tensor(x.numpy()).shape

In [ ]:
blob[0]['FRONT']['labels'].labels[0]

In [ ]:
blob[0]['FRONT']['labels'].labels[2].box.center_x

In [ ]:
import os
import time
import tensorflow.compat.v1 as tf
import pickle

tf.enable_eager_execution() # No need for session to be created. Function instances are run immediately. 

from waymo_open_dataset import dataset_pb2 as open_dataset
from google.cloud import storage

import concurrent.futures as concurr

# CONFIG
project = "Waymo3DObjectDetection"
bucket_name = 'waymo_open_dataset_v_1_2_0_individual_files'
suffix = '.tfrecord'
data_destination = os.getcwd() + "/data/"
download_batch_size = 1

def download_blob(blob, c):
    """
    blob = single file name
    c = file counter
    """
    fname = f"{data_destination}blob_{c}{suffix}"
    blob.download_to_filename(fname)
    return fname

In [ ]:
# Initialise a client
storage_client = storage.Client(project= project) #storage.Client(project= "Waymo3DObjectDetection", credentials=credentials)
# Create a bucket object for our bucket
bucket = storage_client.get_bucket(bucket_name)
# Get blob files in bucket
blobs = [blob for blob in storage_client.list_blobs(bucket_name, prefix='training/')]

n_blobs = len(blobs) # Number of blobs in the training dataset
print(f'Total number of blobs is {n_blobs}')

In [ ]:
download_blob(blobs[0], 0)

In [ ]:
downloaded_blob = '/home/project_x/data/blob_0.tfrecord'
dataset = tf.data.TFRecordDataset(downloaded_blob, compression_type='')
frames = []
for idx, data in enumerate(dataset):
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    print(len(frame.camera_labels))
    frames.append(frame)
    # Function to strip away LIDAR and other garbage from frame
#     break

In [ ]:
print(frame.camera_labels[4])

In [ ]:
for i in frames:
    print(i.camera_labels[5].name)
        for i, camera in enumerate(["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]):

"blob_1_frame_19"

In [ ]:
[1,2,3,4,5,6 ,7][:]

In [ ]:
200*800/1000

In [ ]:
mkdir FRONT
mkdir SIDE_RIGHT
mkdir FRONT_LEFT
mkdir FRONT_RIGHT
mkdir SIDE_LEFT


In [ ]:
os.listdir('data/training/FRONT')

In [2]:
import os
import torch
from torch.utils.data import Dataset
import pickle
import tensorflow.compat.v1 as tf
CAMERAS = ["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]
BIG_X = 1280
SMALL_X = 886
Y_DIM = 1920

class WaymoDataset(Dataset):
    """Dataset for image segmentation and regression."""

    def __init__(self, scope='training', cameras=CAMERAS, order='random', exclusions=None, heatmaps=True):
        
        self.heatmaps = heatmaps

        # Create list of all filepaths
        self.filepaths = []
        root_path = os.getcwd() + f'/data/{scope}'
        for cam in CAMERAS:
            cam_filepaths = os.listdir(f'{root_path}/{cam}')
            self.filepaths += [f'{root_path}/{cam}/{i}' for i in cam_filepaths]

        # Filter exclusions
        if exclusions is not None:
            self.filepaths = [i for i in self.filepaths if not any(j in i for j in exclusions)]

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, item):
        
        fpath = self.filepaths[item]
        sample = load_pickle(fpath)
        img = torch.tensor(
            tf.image.decode_jpeg(sample['image']).numpy())

        cam_type = fpath.split('/')[5] # FRONT, SIDE_LEFT etc
        labels = scale_labels(sample['labels'].labels, cam_type)
        if self.heatmaps:
            labels = convert_to_heatmap(labels)

        return {'img': img, 'labels' : labels}


    def get_context(self, item):
        return self.img_names[item]

def scale_labels(labels, cam='FRONT'):
    if cam in ['SIDE_LEFT', 'SIDE_RIGHT']:
        x_dim = BIG_X
    else:
        x_dim = SMALL_X
    res = {i+1:[] for i in range(4)}
    for i in labels:
        res[i.type].append({
            'id': i.id,
            'x': i.box.center_x/x_dim,
            'y': i.box.center_y/Y_DIM,
            'width': i.box.width/x_dim,
            'length': i.box.length/Y_DIM
        })
    return res
def load_pickle(fpath):
    with open(fpath, 'rb') as f:
        return pickle.load(f)

def convert_to_heatmap(img_dims, labels):
    # TODO
    pass

wD = WaymoDataset(cameras=['FRONT'], heatmaps=False)


In [5]:
loader = torch.utils.data.DataLoader(wD,
                                    batch_size=4, shuffle=True,
                                    um_workers=4)

In [16]:
x = load_pickle(wD.filepaths[0])['labels']
x

name: FRONT
labels {
  box {
    center_x: 916.963565864244
    center_y: 689.1694420615702
    width: 18.00022448145353
    length: 15.57915965430243
  }
  type: TYPE_VEHICLE
  id: "088fc0a9-a56e-46f1-b975-13b557c951df"
}
labels {
  box {
    center_x: 608.4022651186859
    center_y: 773.8779320734784
    width: 103.30981203028102
    length: 139.4908225918482
  }
  type: TYPE_VEHICLE
  id: "3207b332-ca3b-4e67-850a-7572a023ed5e"
}
labels {
  box {
    center_x: 1123.2820572662786
    center_y: 749.0652348441859
    width: 104.21235000000001
    length: 124.10728345325583
  }
  type: TYPE_VEHICLE
  id: "4438e317-45d5-4318-880d-928f72d6ae80"
}
labels {
  box {
    center_x: 903.489533682083
    center_y: 689.6962247398812
    width: 20.210879999999975
    length: 24.947810526011835
  }
  type: TYPE_VEHICLE
  id: "46be34fb-7879-4755-a73f-03e5339592f5"
}
labels {
  box {
    center_x: 862.2467139762578
    center_y: 702.833288340566
    width: 35.36905768317604
    length: 45.600790926729